In [2]:
import keras
import numpy as np 
import pandas as pd
from hyperopt import STATUS_OK, Trials,fmin,hp,tpe
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from urllib.parse import urlparse


import mlflow
from mlflow.models import infer_signature

C:\Users\anmol\Desktop\ML_Starter\venv\Lib\site-packages\hyperopt\atpe.py:19: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [3]:
## LOAD THE DATASET

data = pd.read_csv("https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-white.csv", sep=";")

data

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
...,...,...,...,...,...,...,...,...,...,...,...,...
4893,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6
4894,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5
4895,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6
4896,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7


In [4]:
## SPLIT THE DATA INTO TRAINING, VALIDATION AND TEST SETS

train, test = train_test_split(data, test_size=0.25,random_state=42)
train


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
2835,6.3,0.25,0.22,3.30,0.048,41.0,161.0,0.99256,3.16,0.50,10.5,6
1157,7.8,0.30,0.29,16.85,0.054,23.0,135.0,0.99980,3.16,0.38,9.0,6
744,7.4,0.38,0.27,7.50,0.041,24.0,160.0,0.99535,3.17,0.43,10.0,5
1448,7.4,0.16,0.49,1.20,0.055,18.0,150.0,0.99170,3.23,0.47,11.2,6
3338,7.2,0.27,0.28,15.20,0.046,6.0,41.0,0.99665,3.17,0.39,10.9,6
...,...,...,...,...,...,...,...,...,...,...,...,...
4426,6.2,0.21,0.52,6.50,0.047,28.0,123.0,0.99418,3.22,0.49,9.9,6
466,7.0,0.14,0.32,9.00,0.039,54.0,141.0,0.99560,3.22,0.43,9.4,6
3092,7.6,0.27,0.52,3.20,0.043,28.0,152.0,0.99129,3.02,0.53,11.4,6
3772,6.3,0.24,0.29,13.70,0.035,53.0,134.0,0.99567,3.17,0.38,10.6,6


In [5]:
print(train.columns)

Index(['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol', 'quality'],
      dtype='object')


In [6]:
train_x = train.drop(['quality'], axis=1).values
train_y = train[['quality']].values.ravel()

## VALIDATION DATASET
test_x = test.drop(['quality'], axis=1).values
test_y = test[['quality']].values.ravel()

## SPLITTING THIS TRAIN DATA INTO TRAIN AND VALIDATION 
train_x, valid_x, train_y, valid_y = train_test_split(train_x, train_y, test_size=0.20, random_state=42)

signature = infer_signature(train_x, train_y)

In [7]:
### ANN Model

def train_model(params, epochs, train_x, train_y, valid_x, valid_y, test_x, test_y):

    ## Define model architecture
    mean  = np.mean(train_x, axis=0)
    var = np.var(train_x, axis=0)

    model = keras.Sequential(
        [
            keras.Input([train_x.shape[1]]),
            keras.layers.Normalization(mean=mean, variance= var),
            keras.layers.Dense(64, activation='relu'),
            keras.layers.Dense(1)
        ]
    )

    ## COMPILE THE MODEL

    model.compile(optimizer=keras.optimizers.SGD(
        learning_rate = params["lr"], momentum=params["momentum"]
    ),
    loss="mean_squared_error",
    metrics=[keras.metrics.RootMeanSquaredError()]
                 )

    ## TRAIN THE ANN MODEL with lr and momentum params with MLFLOW Tracking

    with mlflow.start_run(nested=True):
        model.fit(train_x, train_y, validation_data=(valid_x, valid_y),epochs=epochs, batch_size = 64)

        ## EVALUATE THE MODEL
        eval_result=model.evaluate(valid_x, valid_y, batch_size = 64)
        eval_rmse = eval_result[1]

        ## LOG THE MODEL PARAMS AND RESULTS
        mlflow.log_params(params)
        mlflow.log_metric("eval_rmse", eval_rmse)

        ## LOG THE MODEL
        mlflow.tensorflow.log_model(model,"model",signature=signature)

        return {"loss": eval_rmse, "status": STATUS_OK, "model": model}
    

In [8]:
def objective(params):
    #MLFLOW will track the parameters and results for each run

    result = train_model(
        params, 
        epochs=3, 
        train_x = train_x, 
        train_y = train_y, 
        valid_x = valid_x,
        valid_y = valid_y,
        test_x = test_x,
        test_y = test_y,
    )

    return result

In [9]:
space={
    "lr": hp.loguniform("lr",np.log(1e-5),np.log(1e-1)),
    "momentum":hp.uniform("momentum",0.0, 1.0)
}

In [11]:
mlflow.set_tracking_uri(uri="http://127.0.0.1:5000")

# 2. Optional: Check the tracking URI scheme
tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

mlflow.set_experiment("/wine-quality")
with mlflow.start_run():
    # CONDUCT THE HYPERPARAMETER SEARCH USING Hyperopt
    trials= Trials()
    best=fmin(
        fn=objective,
        space=space, 
        algo = tpe.suggest,
        max_evals=4,
        trials=trials
    )

    ## FETCH THE DETAILS OF THE BEST RUN
    best_run = sorted(trials.results, key=lambda x:x["loss"])[0]

    ## LOG THE BEST PARAMETER, LOSS AND MODEL

    mlflow.log_params(best)
    mlflow.log_metric("eval_rmse", best_run["loss"])
    mlflow.tensorflow.log_model(best_run["model"], "model", signature = signature)

    ## PRINT OUT THE BEST PARAMETERS AND CORRESPONDING LOSS
    print(f"Best Parameters: {best}")
    print(f"Best eval rmse: {best_run['loss']}")

2025/06/07 21:54:04 INFO mlflow.tracking.fluent: Experiment with name '/wine-quality' does not exist. Creating a new experiment.


Epoch 1/3                                            

 1/46 ━━━━━━━━━━━━━━━━━━━━ 50s 1s/step - loss: 40.5066 - root_mean_squared_error: 6.3645
27/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 16.2928 - root_mean_squared_error: 3.9348
46/46 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - loss: 12.7176 - root_mean_squared_error: 3.4556 - val_loss: 4.0941 - val_root_mean_squared_error: 2.0234

Epoch 2/3                                            

 1/46 ━━━━━━━━━━━━━━━━━━━━ 2s 54ms/step - loss: 3.6123 - root_mean_squared_error: 1.9006
23/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 9.0526 - root_mean_squared_error: 2.9470 
42/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 9.4569 - root_mean_squared_error: 3.0384
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 9.7274 - root_mean_squared_error: 3.0828 - val_loss: 25.5942 - val_root_mean_squared_error: 5.0591

Epoch 3/3                                            

 1/46 ━━━━━━━━━━━━━━━━━━━━ 5s 123ms/step - loss: 49.2934 - root_mean_squared_error: 7.0209
10/46 